# Sensitivity across cancer cell lines
We evaluated catalogued relationships in MOAlmanac across cancer cell lines. This notebook should be run from the MOAlmanac directory. 

In [1]:
import pandas as pd
import tinydb

almanac = tinydb.TinyDB('/Users/brendan/Github/moalmanac/moalmanac/datasources/almanac/almanac.0.4.7.json')
almanac_genes = almanac.table('genes').all()[0]['genes']

In [2]:
variant = pd.read_csv('datasources/preclinical/ccle.variants.evaluated.txt', sep='\t')
cna = pd.read_csv('datasources/preclinical/ccle.copy-numbers.evaluated.txt', sep='\t')
fusion = pd.read_csv('datasources/preclinical/sanger.fusions.evaluated.txt', sep='\t')

variant = variant.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
variant['feature_type'] = 'Somatic Variant'
variant = variant[variant['feature'].isin(almanac_genes)].reset_index(drop=True)

cna = cna.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
cna['feature_type'] = 'Copy Number'
cna = cna[cna['feature'].isin(almanac_genes)].reset_index(drop=True)

fusion = fusion.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
fusion['feature_type'] = 'Rearrangement'
fusion = fusion[fusion['feature'].isin(almanac_genes)].reset_index(drop=True)

In [3]:
%load_ext autoreload
%autoreload 2

from config import COLNAMES
from config import CONFIG

from annotator import Annotator, Almanac
from datasources import Datasources
from features import Features

def annotate_somatic(df, dbs, ontology='Unknown'):
    df[Annotator.score_bin] = Annotator.preallocate_bin(Annotator.score_bin, df.index)
    df = Almanac.annotate(df, dbs, ontology)
    return df

dbs = Datasources.generate_db_dict(CONFIG)
    
variant = Features.preallocate_missing_columns(variant)
annotated_variant = annotate_somatic(variant, dbs)

In [4]:
cna = Features.preallocate_missing_columns(cna)
annotated_cna = annotate_somatic(cna, dbs)

In [5]:
fusion = Features.preallocate_missing_columns(fusion)
annotated_fusion = annotate_somatic(fusion, dbs)

In [17]:
annotated_somatic = pd.concat([annotated_variant, annotated_cna, annotated_fusion], ignore_index=True)
annotated_somatic.to_csv('annotated_somatic.ccle.txt', sep='\t')

In [16]:
%load_ext autoreload
%autoreload 2

import evaluator

In [21]:
def evaluate_somatic(df):
    df = evaluator.Evaluator.evaluate_almanac(df)
    return df

evaluated_somatic = evaluate_somatic(annotated_somatic)

In [24]:
evaluated_germline = Features.create_empty_dataframe()
evaluated_ms_variants = Features.create_empty_dataframe()
evaluated_ms_status = Features.create_empty_dataframe()
evaluated_burden = Features.create_empty_dataframe()
evaluated_burden.loc[0, 'a'] = ''
evaluated_signatures = Features.create_empty_dataframe()
evaluated_wgd = Features.create_empty_dataframe()

actionable = evaluator.Actionable.evaluate(evaluated_somatic, 
                                           evaluated_germline,
                                           evaluated_ms_variants,
                                           evaluated_ms_status,
                                           evaluated_burden,
                                           evaluated_signatures,
                                           evaluated_wgd
                                          )

In [27]:
%load_ext autoreload
%autoreload 2

import annotator
import datasources
import investigator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
actionable['alteration'].fillna('', inplace=True)

In [101]:
idx = actionable[actionable['feature_type'].eq('Rearrangement') & (actionable['alteration'].eq('') | actionable['alteration'].eq('None--None'))].index
actionable.loc[idx, 'alteration'] = actionable.loc[idx, 'feature'] + '--None'

In [102]:
actionable[actionable['feature_type'].eq('Rearrangement')]

,feature,alteration_type,alteration,feature_type,almanac_bin,feature_match_3,resistance_source_type,prognostic_feature_type,connections,mut_ic50_median,...,wt_ic50_std,exac_sas_an,preferred_assertion,prognostic_oncotree_term,exac_eas_ac,exac_ac,sensitive_score_bin_map,resistance_score_bin_map,prognostic_score_bin_map,a
8705,EWSR1,Fusion,EWSR1--None,Rearrangement,3,,,Rearrangement,,,...,,,,Ewing Sarcoma,,,NaN,NaN,NaN,NaN
8698,FGFR2,Fusion,FGFR2--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8699,ALK,Fusion,ALK--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8700,ROS1,Fusion,ROS1--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8701,BCR,Fusion,BCR--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8702,FGFR3,Fusion,FGFR3--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8695,EML4,Fusion,EML4--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8696,COL1A1,Fusion,COL1A1--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8690,RUNX1,Fusion,RUNX1--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN
8693,ESRP1,Fusion,ESRP1--None,Rearrangement,3,,,,,,...,,,,,,,NaN,NaN,NaN,NaN


In [117]:
#dbs_preclinical = datasources.Preclinical.import_dbs()
efficacy_dictionary = investigator.SensitivityDictionary.create(dbs_preclinical, actionable, 'cell lines')
efficacy_summary = investigator.SummaryDataFrame.create(efficacy_dictionary, actionable, 'cell lines')
#annotated_actionable = annotator.PreclinicalEfficacy.annotate(actionable, efficacy_summary)

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

...

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 7.5))
/Users/brendan/Github/moalmanac/moalmanac/illustrator.py:260: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (

In [125]:
efficacy_summary_all = efficacy_summary.copy(deep=True)
efficacy_summary = efficacy_summary[~efficacy_summary['tested_subfeature'].str.contains("None")]

In [126]:
efficacy_summary.to_csv('cell-lines.efficacy-summary.txt', sep='\t', index=False)

In [127]:
annotated_actionable.to_csv('cell-lines.annotated.actionable.txt', sep='\t', index=False)

In [104]:
efficacy_summary['pvalue_mww'].astype(float).le(0.05).value_counts()

False    4729
True     1659
Name: pvalue_mww, dtype: int64

In [128]:
efficacy_summary['pvalue_mww_float'] = efficacy_summary['pvalue_mww'].astype(float)
tmp = (efficacy_summary
 .loc[:, ['feature_display', 'therapy_name', 'pvalue_mww_float']]
 .sort_values('pvalue_mww_float')
 #.drop_duplicates(['feature_display', 'therapy_name'])
)

tmp[tmp['pvalue_mww_float'].lt(0.05)]#.index

,feature_display,therapy_name,pvalue_mww_float
1468,NRAS p.C186F (Missense),Selumetinib,0.000
85,PIK3CA p.E545G (Missense),Pictilisib,0.000
5726,EGFR Deletion,Gefitinib,0.000
2519,PIK3CA p.E726K (Missense),Pictilisib,0.000
5600,NRAS p.T20fs (Frameshift),Selumetinib,0.000
...,...,...,...
5589,BRCA2 Amplification,Olaparib,0.042
1446,BRCA2 Deletion,Olaparib,0.042
4557,BRCA2 p.SK2978fs (Frameshift),Olaparib,0.044
31,KRAS p.G12C (Missense),KRAS (G12C) Inhibitor-12,0.048


In [129]:
tmp_sig = (efficacy_summary
 .loc[tmp[tmp['pvalue_mww_float'].lt(0.05)].index, :]
 .loc[:, ['tested_subfeature', 'therapy_name', 'pvalue_mww_float']]
 .drop_duplicates(['tested_subfeature', 'therapy_name'])
)

In [130]:
tmp_sig['therapy_name'].drop_duplicates().shape

(18,)

In [134]:
tmp_sig[tmp_sig['tested_subfeature'].str.contains('p.D594G')]

,tested_subfeature,therapy_name,pvalue_mww_float


In [139]:
efficacy_summary[efficacy_summary['feature_display'].str.contains('D594G')]

,patient_id,feature_display,tested_subfeature,therapy_name,n_mut,n_wt,n_mut_tested,n_wt_tested,mut_ic50_median,mut_ic50_mean,mut_ic50_std,wt_ic50_median,wt_ic50_mean,wt_ic50_std,pvalue_mww,statistic,pvalue_mww_float


In [132]:
efficacy_summary.loc[5710, :]

patient_id                                        cell lines
feature_display                    ERRFI1 p.L145V (Missense)
tested_subfeature    ERRFI1 Somatic Variant Missense p.L145V
therapy_name                                       Erlotinib
n_mut                                                      1
n_wt                                                    1822
n_mut_tested                                               0
n_wt_tested                                              752
mut_ic50_median                                          NaN
mut_ic50_mean                                            NaN
mut_ic50_std                                             NaN
wt_ic50_median                                         13.41
wt_ic50_mean                                          24.421
wt_ic50_std                                           34.777
pvalue_mww                                               nan
statistic                                                NaN
pvalue_mww_float        

In [106]:
efficacy_summary['']

,patient_id,feature_display,tested_subfeature,therapy_name,n_mut,n_wt,n_mut_tested,n_wt_tested,mut_ic50_median,mut_ic50_mean,mut_ic50_std,wt_ic50_median,wt_ic50_mean,wt_ic50_std,pvalue_mww,statistic
0,cell lines,BRAF p.V600E (Missense),BRAF,Dabrafenib,264,1559,136,621,34.824,120.732,228.705,121.915,244.298,422.653,2.322e-12,58428
1,cell lines,BRAF p.V600E (Missense),BRAF Somatic Variant,Dabrafenib,184,1639,98,659,19.092,80.76,176.13,120.697,243.118,416.712,7.627e-17,49130
2,cell lines,BRAF p.V600E (Missense),BRAF Somatic Variant Missense,Dabrafenib,170,1653,89,668,19.04,82.289,184.013,119.74,240.726,414.446,6.330e-16,45390
3,cell lines,BRAF p.V600E (Missense),BRAF Somatic Variant Missense p.V600E,Dabrafenib,101,1722,49,708,3.725,16.282,39.616,118.217,236.343,407.192,5.518e-23,31960
4,cell lines,BRAF p.V600E (Missense),BRAF,Trametinib,264,1559,137,634,0.516,5.862,27.545,2.112,7.605,32.168,2.344e-09,57546.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,cell lines,CCND1 Deletion,CCND1 Copy Number,Palbociclib,123,1700,71,699,83.568,116.447,117.936,36.387,108.191,265.377,0.002,19414.5
6384,cell lines,CCND1 Deletion,CCND1 Copy Number Deletion,Palbociclib,5,1818,2,768,23.433,23.433,19.501,39.039,109.175,255.627,0.457,1002
6385,cell lines,CDKN2C Amplification,CDKN2C,Palbociclib,67,1756,34,736,49.318,71.056,90.922,38.158,110.703,260.326,0.661,11956
6386,cell lines,CDKN2C Amplification,CDKN2C Copy Number,Palbociclib,51,1772,23,747,48.388,71.104,98.045,38.414,110.118,258.603,0.758,8266
